In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime, timedelta
np.random.seed(1)

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import random
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
ks_2016_dataset = pd.read_csv('./data_2016/ks_2016.csv')

In [5]:
ks_2016_dataset['state'].value_counts()

successful    14250
failed        13163
Name: state, dtype: int64

In [7]:
url_ks_2016 = ks_2016_dataset['url_project']
url_ks_2016

0        https://www.kickstarter.com/projects/patshand/...
1        https://www.kickstarter.com/projects/micheleko...
2        https://www.kickstarter.com/projects/906000710...
3        https://www.kickstarter.com/projects/wspublich...
4        https://www.kickstarter.com/projects/studr/stu...
                               ...                        
27408    https://www.kickstarter.com/projects/ailian/10...
27409    https://www.kickstarter.com/projects/202171438...
27410    https://www.kickstarter.com/projects/121034010...
27411    https://www.kickstarter.com/projects/visunique...
27412    https://www.kickstarter.com/projects/135999638...
Name: url_project, Length: 27413, dtype: object

In [10]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = 'ks_content_crawled_2016.csv'
filename_errors = 'ks_middle_errors_2016.csv'
content_list = []

for idx, url in enumerate(url_ks_2016[23815:], start=23815):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        
        #오류 나기 전까지 저장되어 있는 content_list에는 이어서 append하고; 그래서 content_list에 모으는 것도 유지할 수 있도록
        content_list.append([idx, contents_collected, risk_challenge])
        
        #오류 난 후부턴 다시 크롤링 할 때, 매 페이지 긁어온 것마다 바로바로 csv파일에 한줄씩 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)
    
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 4:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list.append([idx, contents_collected, risk_challenge])
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print(idx)
                break
            
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        #만약 재시도를 10번 했는데도 크롤링에 실패한다면, 원래 했던대로 오류 index, url, nan값을 csv파일들에 저장
        if count == 4:
            content_list.append([idx, error_url, np.nan])
            
            error_into_content_csv = []
            error_into_content_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_content_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
        
        continue

driver.close()

23815
23816
23817
23818
23819
23820
23821
23822
23823
23824
23825
23826
23827
23828
23829
23830
23831
23832
23833
23834
23835
23836
23837
23838
23839
23840
23841
23842
23843
23844
23845
23846
23847
23848
23849
23850
23851
23852
23853
23854
23855
23856
23857
23858
23859
23860
23861
23862
23863
23864
23865
23866
23867
23868
23869
23870
23871
23872
23873
23874
23875
23876
23877
23878
23879
23880
23881
23882
23883
23884
23885
23886
23887
23888
23889
23890
23891
23892
23893
23894
Unexpected error at 23895 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 3
23895
23896
23897
23898
23899
23900
23901
23902
23903
23904
23905
23906
23907
23908
23909
23910
23911
23912
23913
23914
23915
23916
23917
23918
23919
23920
23921
23922
23923
23924
23925
23926
23927
23928


In [ ]:
#5000-9000 추합

In [14]:
c1 = pd.read_csv('./data_2016/content_2016_5000_10000/ks_content_crawled_2016_5.csv')

In [15]:
c2 = pd.read_csv('./data_2016/content_2016_5000_10000/ks_content_crawled_2016_6.csv')

In [16]:
c3 = pd.read_csv('./data_2016/content_2016_5000_10000/ks_content_crawled_2016_7.csv')

In [17]:
c4 = pd.read_csv('./data_2016/content_2016_5000_10000/ks_content_crawled_2016_8.csv')

In [18]:
c5 = pd.read_csv('./data_2016/content_2016_5000_10000/ks_content_crawled_2016_9000_10000.csv')

In [21]:
concat_crawled_2016 = pd.concat([c1,c2,c3,c4,c5], ignore_index=True)
concat_crawled_2016

index                                            content  \
0      5000  ['Turn your waste into a city wide art install...   
1      5001  ['Over the years, we have been looking for the...   
2      5002  ["SUPPORT one band's effort to balance parenti...   
3      5003  ['The definition of the word "inflorescence" i...   
4      5004  ['Alex Rowe (Jeff Ellenberger)\xa0is a black o...   
...     ...                                                ...   
4994   9995  ['All kids deserve access to high quality educ...   
4995   9996  ['The original Legion TD went beyond our wilde...   
4996   9997  ['Hello all. After the success of the RHLSTP k...   
4997   9998  ['', 'Dmitrij Schurbin about HelloArtist', '',...   
4998   9999  ['', '', '|\xa0Facebook\xa0 \xa0| Twitter\xa0 ...   

                                         risk_challenge  
0     You might ask how to produce a project of such...  
1     Risks and challenges are a part of every busin...  
2     As a group, The Savage Hearts is a new band, h...  
3     I assure you that I have the skills to make yo...  
4     Every film production comes with a certain lev...  
...                                                 ...  
4994  After the project is funded, and the upgraded ...  
4995  We're aware of the many risks & challenges tha...  
4996  We need to make sure that AIOTM <aiotm> is as ...  
4997  During the last 3 years we have studied our ta...  
4998  As we are already a professional publishing ho...  

[4999 rows x 3 columns]

In [ ]:
#1개가 빈다 - 6999 없음

In [24]:
concat_crawled_2016[concat_crawled_2016['index'] == 6999]

Empty DataFrame
Columns: [index, content, risk_challenge]
Index: []

In [37]:
indexed_df = concat_crawled_2016.set_index('index')
indexed_df

content  \
index                                                      
5000   ['Turn your waste into a city wide art install...   
5001   ['Over the years, we have been looking for the...   
5002   ["SUPPORT one band's effort to balance parenti...   
5003   ['The definition of the word "inflorescence" i...   
5004   ['Alex Rowe (Jeff Ellenberger)\xa0is a black o...   
...                                                  ...   
9995   ['All kids deserve access to high quality educ...   
9996   ['The original Legion TD went beyond our wilde...   
9997   ['Hello all. After the success of the RHLSTP k...   
9998   ['', 'Dmitrij Schurbin about HelloArtist', '',...   
9999   ['', '', '|\xa0Facebook\xa0 \xa0| Twitter\xa0 ...   

                                          risk_challenge  
index                                                     
5000   You might ask how to produce a project of such...  
5001   Risks and challenges are a part of every busin...  
5002   As a group, The Savage Hearts is a new band, h...  
5003   I assure you that I have the skills to make yo...  
5004   Every film production comes with a certain lev...  
...                                                  ...  
9995   After the project is funded, and the upgraded ...  
9996   We're aware of the many risks & challenges tha...  
9997   We need to make sure that AIOTM <aiotm> is as ...  
9998   During the last 3 years we have studied our ta...  
9999   As we are already a professional publishing ho...  

[4999 rows x 2 columns]

In [43]:
indexed_df.loc[6999] = [np.nan, np.nan] #이것도 가능: indexed_df.loc[6999] = {'content':np.nan, 'risk_challenge':np.nan}

In [ ]:
# 6999는 일단 nan값으로 채워 넣었다

In [44]:
indexed_df

content  \
index                                                      
5000   ['Turn your waste into a city wide art install...   
5001   ['Over the years, we have been looking for the...   
5002   ["SUPPORT one band's effort to balance parenti...   
5003   ['The definition of the word "inflorescence" i...   
5004   ['Alex Rowe (Jeff Ellenberger)\xa0is a black o...   
...                                                  ...   
9996   ['The original Legion TD went beyond our wilde...   
9997   ['Hello all. After the success of the RHLSTP k...   
9998   ['', 'Dmitrij Schurbin about HelloArtist', '',...   
9999   ['', '', '|\xa0Facebook\xa0 \xa0| Twitter\xa0 ...   
6999                                                 NaN   

                                          risk_challenge  
index                                                     
5000   You might ask how to produce a project of such...  
5001   Risks and challenges are a part of every busin...  
5002   As a group, The Savage Hearts is a new band, h...  
5003   I assure you that I have the skills to make yo...  
5004   Every film production comes with a certain lev...  
...                                                  ...  
9996   We're aware of the many risks & challenges tha...  
9997   We need to make sure that AIOTM <aiotm> is as ...  
9998   During the last 3 years we have studied our ta...  
9999   As we are already a professional publishing ho...  
6999                                                 NaN  

[5000 rows x 2 columns]

In [47]:
indexed_df.sort_index(inplace=True)

In [48]:
indexed_df

content  \
index                                                      
5000   ['Turn your waste into a city wide art install...   
5001   ['Over the years, we have been looking for the...   
5002   ["SUPPORT one band's effort to balance parenti...   
5003   ['The definition of the word "inflorescence" i...   
5004   ['Alex Rowe (Jeff Ellenberger)\xa0is a black o...   
...                                                  ...   
9995   ['All kids deserve access to high quality educ...   
9996   ['The original Legion TD went beyond our wilde...   
9997   ['Hello all. After the success of the RHLSTP k...   
9998   ['', 'Dmitrij Schurbin about HelloArtist', '',...   
9999   ['', '', '|\xa0Facebook\xa0 \xa0| Twitter\xa0 ...   

                                          risk_challenge  
index                                                     
5000   You might ask how to produce a project of such...  
5001   Risks and challenges are a part of every busin...  
5002   As a group, The Savage Hearts is a new band, h...  
5003   I assure you that I have the skills to make yo...  
5004   Every film production comes with a certain lev...  
...                                                  ...  
9995   After the project is funded, and the upgraded ...  
9996   We're aware of the many risks & challenges tha...  
9997   We need to make sure that AIOTM <aiotm> is as ...  
9998   During the last 3 years we have studied our ta...  
9999   As we are already a professional publishing ho...  

[5000 rows x 2 columns]

In [50]:
concat_crawled_2016 = indexed_df.reset_index()
concat_crawled_2016

index                                            content  \
0      5000  ['Turn your waste into a city wide art install...   
1      5001  ['Over the years, we have been looking for the...   
2      5002  ["SUPPORT one band's effort to balance parenti...   
3      5003  ['The definition of the word "inflorescence" i...   
4      5004  ['Alex Rowe (Jeff Ellenberger)\xa0is a black o...   
...     ...                                                ...   
4995   9995  ['All kids deserve access to high quality educ...   
4996   9996  ['The original Legion TD went beyond our wilde...   
4997   9997  ['Hello all. After the success of the RHLSTP k...   
4998   9998  ['', 'Dmitrij Schurbin about HelloArtist', '',...   
4999   9999  ['', '', '|\xa0Facebook\xa0 \xa0| Twitter\xa0 ...   

                                         risk_challenge  
0     You might ask how to produce a project of such...  
1     Risks and challenges are a part of every busin...  
2     As a group, The Savage Hearts is a new band, h...  
3     I assure you that I have the skills to make yo...  
4     Every film production comes with a certain lev...  
...                                                 ...  
4995  After the project is funded, and the upgraded ...  
4996  We're aware of the many risks & challenges tha...  
4997  We need to make sure that AIOTM <aiotm> is as ...  
4998  During the last 3 years we have studied our ta...  
4999  As we are already a professional publishing ho...  

[5000 rows x 3 columns]

In [51]:
concat_crawled_2016.isna().sum() #결측치 1개는 index 6999

index              0
content            1
risk_challenge    17
dtype: int64

In [54]:
concat_crawled_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
index             5000 non-null int64
content           4999 non-null object
risk_challenge    4983 non-null object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [84]:
concat_crawled_2016.to_csv('./data_2016/ks_content_crawled_2016_5000_10000.csv', index=False, encoding='utf-8')

In [ ]:
#오류 났던 부분들 찾기

In [59]:
errors_df = concat_crawled_2016[concat_crawled_2016['content'].str.startswith('http') == True]
errors_df

index                                            content risk_challenge
1218   6218  https://www.kickstarter.com/projects/jessowen/...            NaN
1325   6325  https://www.kickstarter.com/projects/278818747...            NaN
1431   6431  https://www.kickstarter.com/projects/yenniyen/...            NaN
1655   6655  https://www.kickstarter.com/projects/563115656...            NaN
1767   6767  https://www.kickstarter.com/projects/165569536...            NaN
1887   6887  https://www.kickstarter.com/projects/351718576...            NaN
1998   6998  https://www.kickstarter.com/projects/995602129...            NaN
2128   7128  https://www.kickstarter.com/projects/879623758...            NaN
2239   7239  https://www.kickstarter.com/projects/189646937...            NaN
2355   7355  https://www.kickstarter.com/projects/574090994...            NaN
2459   7459  https://www.kickstarter.com/projects/198721926...            NaN
2565   7565  https://www.kickstarter.com/projects/314684966...            NaN
2670   7670  https://www.kickstarter.com/projects/193818977...            NaN
2771   7771  https://www.kickstarter.com/projects/americanf...            NaN
2890   7890  https://www.kickstarter.com/projects/199464474...            NaN
3669   8669  https://www.kickstarter.com/projects/121461725...            NaN

In [ ]:
#8669는 hidden 페이지 --> nan 전환
#6999 url 찾아서 조회해서 존재하는 페이지이면 url 위 df에 추가

In [60]:
ks_2016_dataset = pd.read_csv('./data_2016/ks_2016.csv')

In [62]:
ks_2016_dataset.reset_index(inplace=True)

In [66]:
ks_2016_dataset[ks_2016_dataset['index'] == 6999]['url_project'].values

array(['https://www.kickstarter.com/projects/2078972315/v7-workspace?ref=discovery_category_newest'],
      dtype=object)

In [ ]:
#6999 페이지 존재

In [68]:
errors_df.reset_index(drop=True, inplace=True)

In [71]:
errors_df.loc[16] = [6999, 'https://www.kickstarter.com/projects/2078972315/v7-workspace?ref=discovery_category_newest', np.nan]

C:\Users\Playdata\Anaconda3\envs\tf\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [73]:
errors_df.sort_values(by='index', inplace=True)

C:\Users\Playdata\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
errors_df

index                                            content risk_challenge
0    6218  https://www.kickstarter.com/projects/jessowen/...            NaN
1    6325  https://www.kickstarter.com/projects/278818747...            NaN
2    6431  https://www.kickstarter.com/projects/yenniyen/...            NaN
3    6655  https://www.kickstarter.com/projects/563115656...            NaN
4    6767  https://www.kickstarter.com/projects/165569536...            NaN
5    6887  https://www.kickstarter.com/projects/351718576...            NaN
6    6998  https://www.kickstarter.com/projects/995602129...            NaN
16   6999  https://www.kickstarter.com/projects/207897231...            NaN
7    7128  https://www.kickstarter.com/projects/879623758...            NaN
8    7239  https://www.kickstarter.com/projects/189646937...            NaN
9    7355  https://www.kickstarter.com/projects/574090994...            NaN
10   7459  https://www.kickstarter.com/projects/198721926...            NaN
11   7565  https://www.kickstarter.com/projects/314684966...            NaN
12   7670  https://www.kickstarter.com/projects/193818977...            NaN
13   7771  https://www.kickstarter.com/projects/americanf...            NaN
14   7890  https://www.kickstarter.com/projects/199464474...            NaN
15   8669  https://www.kickstarter.com/projects/121461725...            NaN

In [76]:
errors_df.rename(columns={'content':'error_url'}, inplace=True)

C:\Users\Playdata\Anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [81]:
errors_url = errors_df[['index','error_url']]
errors_url

index                                          error_url
0    6218  https://www.kickstarter.com/projects/jessowen/...
1    6325  https://www.kickstarter.com/projects/278818747...
2    6431  https://www.kickstarter.com/projects/yenniyen/...
3    6655  https://www.kickstarter.com/projects/563115656...
4    6767  https://www.kickstarter.com/projects/165569536...
5    6887  https://www.kickstarter.com/projects/351718576...
6    6998  https://www.kickstarter.com/projects/995602129...
16   6999  https://www.kickstarter.com/projects/207897231...
7    7128  https://www.kickstarter.com/projects/879623758...
8    7239  https://www.kickstarter.com/projects/189646937...
9    7355  https://www.kickstarter.com/projects/574090994...
10   7459  https://www.kickstarter.com/projects/198721926...
11   7565  https://www.kickstarter.com/projects/314684966...
12   7670  https://www.kickstarter.com/projects/193818977...
13   7771  https://www.kickstarter.com/projects/americanf...
14   7890  https://www.kickstarter.com/projects/199464474...
15   8669  https://www.kickstarter.com/projects/121461725...

In [82]:
errors_url.to_csv('./data_2016/ks_middle_errors_2016_5000_10000.csv', index=False, encoding='utf-8')

In [ ]:
#2016년 전체 추합

In [4]:
c1 = pd.read_csv('./data_2016/ks_content_crawled_2016_0_5000.csv')

In [5]:
c2 = pd.read_csv('./data_2016/ks_content_crawled_2016_5000_10000.csv')

In [12]:
c3 = pd.read_csv('./data_2016/ks_content_crawled_2016_10000_15000.csv')

In [7]:
c4 = pd.read_csv('./data_2016/ks_content_crawled_2016_15000_27412.csv')

In [14]:
concat_2016 = pd.concat([c1,c2,c3,c4])

In [15]:
concat_2016.shape

(27413, 3)

In [16]:
ks_2016_dataset.shape

(27413, 49)

In [20]:
concat_2016.sort_values(by='index', inplace=True)

In [22]:
concat_2016.reset_index(drop=True, inplace=True)

In [23]:
concat_2016

index                                            content  \
0          0  ['', '', 'DESTINY, NY is a 120-page graphic no...   
1          1  ['Harnessing a whirlwind of power, intrigue an...   
2          2  ['Logline', 'When a high school student lives ...   
3          3  ['West Salem Public House will exist to create...   
4          4  ['STUDR is an APP that was created to help new...   
...      ...                                                ...   
27408  27408  ['Hi there!', "I'm Ailian. I grew up in Singap...   
27409  27409  ['Fire in my heart started from Personal drama...   
27410  27410  ['"A2Z of Indian Culinary-A photo book with 26...   
27411  27411  ['Ich möchte meine Collage als Printdesign auf...   
27412  27412  ['The Project', 'The Project\nFor my painting ...   

                                          risk_challenge  
0      While this is our first Kickstarter campaign, ...  
1      Every film, like every creative endeavour, has...  
2      As this is a student film, we are anticipating...  
3      Well, no one could come to our doors, but we a...  
4      Well, here's the good news, the APP where it s...  
...                                                  ...  
27408  - The risks are very low. I have printed a ful...  
27409  No Risks!\r\nChallenges ? Only few , And we wo...  
27410  My projects might face unique but very real ch...  
27411  Ich muss nur noch die Druckdaten der Druckerei...  
27412  I am under a tight schedule to execute 15min v...  

[27413 rows x 3 columns]

In [62]:
concat_2016.isna().sum()

index               0
content            13
risk_challenge    128
dtype: int64

In [35]:
errors_2016 = pd.read_csv('./data_2016/ks_middle_errors_2016_0_27412.csv')

In [44]:
errors_2016.shape

(27, 2)

In [42]:
errors_2016['index'].values

array([ 6218,  6325,  6431,  6655,  6767,  6887,  6998,  6999,  7128,
        7239,  7355,  7459,  7565,  7670,  7771,  7890,  8669, 10562,
       11165, 11968, 20213, 20826, 23795, 23796, 23797, 24291, 25704],
      dtype=int64)

In [46]:
content_2016_errors_excluded = concat_2016.drop(errors_2016['index'].values)
content_2016_errors_excluded

index                                            content  \
0          0  ['', '', 'DESTINY, NY is a 120-page graphic no...   
1          1  ['Harnessing a whirlwind of power, intrigue an...   
2          2  ['Logline', 'When a high school student lives ...   
3          3  ['West Salem Public House will exist to create...   
4          4  ['STUDR is an APP that was created to help new...   
...      ...                                                ...   
27408  27408  ['Hi there!', "I'm Ailian. I grew up in Singap...   
27409  27409  ['Fire in my heart started from Personal drama...   
27410  27410  ['"A2Z of Indian Culinary-A photo book with 26...   
27411  27411  ['Ich möchte meine Collage als Printdesign auf...   
27412  27412  ['The Project', 'The Project\nFor my painting ...   

                                          risk_challenge  
0      While this is our first Kickstarter campaign, ...  
1      Every film, like every creative endeavour, has...  
2      As this is a student film, we are anticipating...  
3      Well, no one could come to our doors, but we a...  
4      Well, here's the good news, the APP where it s...  
...                                                  ...  
27408  - The risks are very low. I have printed a ful...  
27409  No Risks!\r\nChallenges ? Only few , And we wo...  
27410  My projects might face unique but very real ch...  
27411  Ich muss nur noch die Druckdaten der Druckerei...  
27412  I am under a tight schedule to execute 15min v...  

[27386 rows x 3 columns]

In [45]:
27413-27

27386

In [47]:
content_2016_errors_excluded.to_csv('./data_2016/ks_content_2016_recrawled_0_27412.csv', index=False, encoding='utf-8')

In [52]:
errors_2016

index                                          error_url
0    6218  https://www.kickstarter.com/projects/jessowen/...
1    6325  https://www.kickstarter.com/projects/278818747...
2    6431  https://www.kickstarter.com/projects/yenniyen/...
3    6655  https://www.kickstarter.com/projects/563115656...
4    6767  https://www.kickstarter.com/projects/165569536...
5    6887  https://www.kickstarter.com/projects/351718576...
6    6998  https://www.kickstarter.com/projects/995602129...
7    6999  https://www.kickstarter.com/projects/207897231...
8    7128  https://www.kickstarter.com/projects/879623758...
9    7239  https://www.kickstarter.com/projects/189646937...
10   7355  https://www.kickstarter.com/projects/574090994...
11   7459  https://www.kickstarter.com/projects/198721926...
12   7565  https://www.kickstarter.com/projects/314684966...
13   7670  https://www.kickstarter.com/projects/193818977...
14   7771  https://www.kickstarter.com/projects/americanf...
15   7890  https://www.kickstarter.com/projects/199464474...
16   8669  https://www.kickstarter.com/projects/121461725...
17  10562  https://www.kickstarter.com/projects/bkamf/2nd...
18  11165  https://www.kickstarter.com/projects/115241568...
19  11968  https://www.kickstarter.com/projects/bullz-i/t...
20  20213  https://www.kickstarter.com/projects/boazvande...
21  20826  https://www.kickstarter.com/projects/193517327...
22  23795  https://www.kickstarter.com/projects/994335254...
23  23796  https://www.kickstarter.com/projects/139942118...
24  23797  https://www.kickstarter.com/projects/15723376/...
25  24291  https://www.kickstarter.com/projects/751790459...
26  25704  https://www.kickstarter.com/projects/sarahdonn...

In [50]:
errors_2016['error_url']

0     https://www.kickstarter.com/projects/jessowen/...
1     https://www.kickstarter.com/projects/278818747...
2     https://www.kickstarter.com/projects/yenniyen/...
3     https://www.kickstarter.com/projects/563115656...
4     https://www.kickstarter.com/projects/165569536...
5     https://www.kickstarter.com/projects/351718576...
6     https://www.kickstarter.com/projects/995602129...
7     https://www.kickstarter.com/projects/207897231...
8     https://www.kickstarter.com/projects/879623758...
9     https://www.kickstarter.com/projects/189646937...
10    https://www.kickstarter.com/projects/574090994...
11    https://www.kickstarter.com/projects/198721926...
12    https://www.kickstarter.com/projects/314684966...
13    https://www.kickstarter.com/projects/193818977...
14    https://www.kickstarter.com/projects/americanf...
15    https://www.kickstarter.com/projects/199464474...
16    https://www.kickstarter.com/projects/121461725...
17    https://www.kickstarter.com/projects/bkamf

In [51]:
errors_2016['index'][0]

6218

In [53]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = './data_2016/ks_content_2016_recrawled_0_27412.csv'
filename_errors = './data_2016/new_errors_recrawled_2016.csv'

for row, url in enumerate(errors_2016['error_url']):
    idx = errors_2016['index'][row]
    
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan

        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print("row:{}".format(row), "index:{}".format(idx))
        
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 2:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print("Retry successful")
                print("row:{}".format(row), "index:{}".format(idx))
                break
                
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        if count == 2:
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
            print("Retry failed")
            
        continue
    
driver.close()

row:0 index:6218
row:1 index:6325
row:2 index:6431
row:3 index:6655
row:4 index:6767
row:5 index:6887
row:6 index:6998
row:7 index:6999
row:8 index:7128
row:9 index:7239
row:10 index:7355
row:11 index:7459
row:12 index:7565
row:13 index:7670
row:14 index:7771
row:15 index:7890
Unexpected error at 8669 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retry failed
row:17 index:10562
Unexpected error at 11165 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retry failed
row:19 index:11968
row:20 index:20213
row:21 index:20826
row:22 index:23795
row:23 index:23796
row:24 index:23797
row:25 index:24291
row:26 index:25704


In [54]:
c_2016 = pd.read_csv('./data_2016/ks_content_2016_recrawled_0_27412.csv')

In [57]:
c_2016.sort_values(by='index', inplace=True)

In [59]:
c_2016.reset_index(drop=True, inplace=True)

In [60]:
c_2016

index                                            content  \
0          0  ['', '', 'DESTINY, NY is a 120-page graphic no...   
1          1  ['Harnessing a whirlwind of power, intrigue an...   
2          2  ['Logline', 'When a high school student lives ...   
3          3  ['West Salem Public House will exist to create...   
4          4  ['STUDR is an APP that was created to help new...   
...      ...                                                ...   
27408  27408  ['Hi there!', "I'm Ailian. I grew up in Singap...   
27409  27409  ['Fire in my heart started from Personal drama...   
27410  27410  ['"A2Z of Indian Culinary-A photo book with 26...   
27411  27411  ['Ich möchte meine Collage als Printdesign auf...   
27412  27412  ['The Project', 'The Project\nFor my painting ...   

                                          risk_challenge  
0      While this is our first Kickstarter campaign, ...  
1      Every film, like every creative endeavour, has...  
2      As this is a student film, we are anticipating...  
3      Well, no one could come to our doors, but we a...  
4      Well, here's the good news, the APP where it s...  
...                                                  ...  
27408  - The risks are very low. I have printed a ful...  
27409  No Risks!\r\nChallenges ? Only few , And we wo...  
27410  My projects might face unique but very real ch...  
27411  Ich muss nur noch die Druckdaten der Druckerei...  
27412  I am under a tight schedule to execute 15min v...  

[27413 rows x 3 columns]

In [61]:
c_2016.isna().sum()

index               0
content             4
risk_challenge    103
dtype: int64

In [63]:
c_2016.to_csv('./data_2016/ks_content_2016_recrawled_0_27412.csv', index=False, encoding='utf-8')